# Notebook to generate input for OrnithoScope

In [147]:
import os, os.path
import pandas as pd
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt
import numpy as np

## Format des données en input
```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

TODO:

Pour chaque objet : 

- Récupérer filename : bird/task_05-01-2021/2021-01-05-15-56-59.jpg
    L'ajouter au path global

- Récupérer le label

- Récupérer les box

In [148]:
import os
import os.path
import pandas as pd
from xml.etree import ElementTree as ET

# Variables
path_annotation = '/home/acarlier/OrnithoMate/p0133_bird_data/annotations/'
path_raw_data = '/home/acarlier/OrnithoMate/p0133_bird_data/raw_data/'
path_input_csv = '/home/acarlier/code/OrnithoScope/data/input.csv'
path_ornithoTasks = '/home/acarlier/code/OrnithoScope/data/Ornithotasks - CVAT_task.csv'
tasks_dir = sorted(os.listdir(path_annotation))

#
unwanted_labels_list = ['unknown', 'human', 'noBird']
columns_input_data = ["split_value", "file_path", "label",
                      "x_min", "y_min", "empty_1", "empty_2",
                      "x_max", "y_max", "empty_3"]
list_object = []
df_input = pd.DataFrame(columns=columns_input_data)

In [149]:
def get_info_from_one_xml(xml_path, list_object):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    df_tasks = import_tasks_as_df()
    # Filename Extraction and local path creation
    filename = root.find("./filename").text
    local_img_path = f"{path_raw_data}{filename.split('/', 1)[1]}"
    # Width and height extraction
    width = int(root.find("./size/width").text)
    height = int(root.find("./size/height").text)
    task_name_xml = filename.split('/')[1]
    # print('task_name', task_name_xml)

    for object in root.findall("./object"):
        # Extract specie name
        for attributes in object.findall("./attributes/"):
            if attributes.find('name').text == 'species':
                specie = attributes.find('value').text
                # Data creation
                data = {}
                data['split_value'] = check_split_value(pd.read_csv(path_ornithoTasks), task_name_xml)
                data['file_path'] = local_img_path
                data['label'] = specie
                # Extract bndbx
                bndbox = object.find('./bndbox')
                x_min = round(float(bndbox.find("xmin").text) / width, 4)
                y_min = round(float(bndbox.find("ymin").text) / height, 4)
                x_max = round(float(bndbox.find("xmax").text) / width, 4)
                y_max = round(float(bndbox.find("ymax").text) / height, 4)
                data['x_min'] = x_min
                data['y_min'] = y_min
                data["empty_1"] = ""
                data["empty_2"] = ""
                data['x_max'] = x_max
                data['y_max'] = y_max
                data["empty_3"] = ""
                list_object.append(data)
    return list_object


def get_info_from_all_xml(tasks_dir, list_object):
    for i in range(len(tasks_dir)):
        # For each task
        task_name = tasks_dir[i]
        complete_path = path_annotation + task_name + '/Annotations/bird/' + task_name + '/'
        # Get all images of a task
        image_paths = sorted(os.listdir(complete_path))
        for img in range(len(image_paths)):
            xml_path = complete_path + image_paths[img]
            get_info_from_one_xml(xml_path, list_object)
            # print(xml_path, 'done')
    return list_object


def import_tasks_as_df():
    df_task = pd.read_csv(path_ornithoTasks)
    df_task = df_task.rename(columns={"Task name": "task_name"})
    return df_task


def check_split_value(df, task_name):
    row_specific_task = df[df['task_name'] == task_name]
    # print('spe task ', row_specific_task)
    val = row_specific_task['Split'].values[0]
    # print('val ', val)
    if val == 'TRAIN':
        return 'TRAINING'
    elif val == 'TEST':
        return 'TEST'
    elif val == 'VALIDATE':
        return 'VALIDATION'


def add_validation_split_value(df):
    return df


def df_to_csv(df, path_and_name_csv):
    return df.to_csv(path_and_name_csv, encoding='utf-8', index=False, header=False)


def csv_to_df(csv_path):
    return pd.DataFrame.to_csv(csv_path)


def create_df_for_input(list_of_object):
    count = 0
    list_df = []
    for object in list_of_object:
        count += 1
        content = pd.DataFrame(object, index=[count])
        list_df.append(content)
    df_input = pd.concat(list_df)
    for label in unwanted_labels_list:
        df_input = df_input[(df_input.label != label)]
    return df_input


def create_input_as_df(task_dir, list_object):
    print('Import data ...')
    get_info_from_all_xml(task_dir, list_object)
    print('Done!')
    print('Create dataframe ...')
    df = create_df_for_input(list_object)
    print('Done!')
    return df


def create_input_as_csv(tasks_dir, list_object, path_input_csv):
    df = create_input_as_df(tasks_dir, list_object)
    df_input = add_validation_split_value(df)
    print('Export to csv ...')
    #df_to_csv(df, path_input_csv)
    print('Done!')


In [150]:
df = create_input_as_df(tasks_dir, list_object)

Import data ...
Done!
Create dataframe ...
Done!


In [151]:
df_to_csv(df, path_input_csv)

In [152]:
(df[df['split_value'] == 'TRAINING']).shape

(10325, 10)

In [153]:
_ = df.copy()

In [154]:
_ = _.reset_index(drop=True)

In [155]:
#_[_['split_value'] == 'TRAINING'].iloc[::10, :] = _[_['split_value'] == 'TRAINING'].iloc[::10, :].replace({"TRAINING":"VALIDATION"})
# _[_.split_value.eq('TRAINING')].iloc[::10, :].replace({"TRAINING":"VALIDATION"})

In [156]:
j = 0
for index, row in _.iterrows():
    if row['split_value'] == 'TRAINING':
        j += 1
        print('j current', j)
        print('split value before ', row['split_value'] )
        if j == 10:
            print(_.split_value.iloc[index])
            #_.split_value.iloc[index] = _.split_value.iloc[index].replace('VALIDATION')
            _.iloc[index, _.columns.get_loc('split_value')] = 'VALIDATION'
            print('split value after ', row['split_value'])
            j = 0

j current 1
split value before  TRAINING
j current 2
split value before  TRAINING
j current 3
split value before  TRAINING
j current 4
split value before  TRAINING
j current 5
split value before  TRAINING
j current 6
split value before  TRAINING
j current 7
split value before  TRAINING
j current 8
split value before  TRAINING
j current 9
split value before  TRAINING
j current 10
split value before  TRAINING
TRAINING
split value after  TRAINING
j current 1
split value before  TRAINING
j current 2
split value before  TRAINING
j current 3
split value before  TRAINING
j current 4
split value before  TRAINING
j current 5
split value before  TRAINING
j current 6
split value before  TRAINING
j current 7
split value before  TRAINING
j current 8
split value before  TRAINING
j current 9
split value before  TRAINING
j current 10
split value before  TRAINING
TRAINING
split value after  TRAINING
j current 1
split value before  TRAINING
j current 2
split value before  TRAINING
j current 3
split valu

In [157]:
_

,split_value,file_path,label,x_min,y_min,empty_1,empty_2,x_max,y_max,empty_3
0,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,MESCHA,0.5558,0.3581,,,0.7055,0.5479,
1,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,SITTOR,0.4613,0.5550,,,0.9990,1.0000,
2,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,MESCHA,0.4402,0.2056,,,0.5508,0.3687,
3,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,SITTOR,0.5870,0.4947,,,0.9899,0.9310,
4,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,MESCHA,0.5286,0.4823,,,1.0000,0.9629,
...,...,...,...,...,...,...,...,...,...,...
14091,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,BERGRI,0.3252,0.4949,,,0.4655,0.6604,
14092,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,BERGRI,0.4208,0.3644,,,0.5614,0.4583,
14093,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,BERGRI,0.3450,0.3715,,,0.6597,0.5682,
14094,TRAINING,/home/acarlier/OrnithoMate/p0133_bird_data/raw...,BERGRI,0.2287,0.4861,,,0.4062,0.6272,


In [158]:
print('TRAIN ', _[_.split_value == 'TRAINING'].shape[0])
print('VALIDATION ', _[_.split_value == 'VALIDATION'].shape[0])
print('TEST ', _[_.split_value == 'TEST'].shape[0])

TRAIN  9293
VALIDATION  1032
TEST  3771
